# The Dice Game
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can play a simple dice game. Here are the basic rules:
* You start with 0 points
* Roll three fair six-sided dice
* Now choose one of the following:
 * Stick, accept the values shown. If two or more dice show the same values, then all of them are flipped upside down: 1 becomes 6, 2 becomes 5, 3 becomes 4, and vice versa. The total is then added to your points and this is your final score.
 * OR reroll the dice. You may choose to hold any combination of the dice on the current value shown. Rerolling costs you 1 point – so during the game and perhaps even at the end your score may be negative. You then make this same choice again.

The best possible score for this game is 18 and is achieved by rolling three 1s on the first roll.

The reroll penalty prevents you from rolling forever to get this score. If the value of the current dice is greater than the expected value of rerolling them (accounting for the penalty), then you should stick.

The optimal decision is independent of your current score. It does not matter whether it is your first roll with a current score of 0, or your twentieth roll with a current score of -19 (in which case a positive end score is impossible), in either of these cases if you roll three 6s (which, if you stick, will only add 3 points) then you still expect to get a *better* end score by rerolling and taking the penalty. Almost any other roll will beat it, so it's still the right choice to maximise your score.

It is pretty obvious that you should stick on three 1s, and reroll on three 6s. Should you hold any of the 6s when you reroll? What about other values? What should you do if the dice come up 3, 4, 5?

We do not know what numbers will come up when we roll, but we do know exactly what the probability of any given roll is. This is the point of the probabilistic reasoning section of the unit; if we can model the true probabilities then we can mathematically calculate the optimal policy. Not all real world situations use dice, but these techniques work well even if we can only estimate the true probabilities.

### Play The Game
You can play the game in the following cell. Change the `SKIP_GAME` constant to `False` to enable this cell. 
<br> *Make sure to change it back to `True` before submitting.*

In [23]:
SKIP_GAME = True
if not SKIP_GAME:
    %run dice_game.py

### Submission Requirements
The code supports playing this game with many possible modifications – you can change the number of dice, the values on the dice, or even make biased (weighted) dice that are more likely to roll certain values. More on this later.

For this assignment you will need to submit:
1. The implementation for an agent which can play the game – this notebook
 * You can write this however you like, using the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned depending on how well your agent plays the game (potentially with modifications)
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests and skeleton code included below, make sure your code is compatible with the format of these tests
2. A PDF that explains your approach and the decisions you made in your own words – a report file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * Your written section must be submitted as a pdf file (.pdf)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section
 
### Choice of Algorithm
You can take any approach you like to write your agent. We have provided two examples for you, though these agents do not perform very well, and would not pass the assignment, they should help you structure your code.

We have covered *value iteration* in the unit, and this technique will work well here to produce a strategy for the game, which your agent can then follow. It is up to you to work out the parameters that will be suit the value iteration algorithm to maximise your score.

However, there are many other possible options. Simply calculating the expected value of a single roll will produce a much stronger strategy than playing randomly. You could also look up various other approaches that can be applied to Markov decision processes, such as policy iteration.

To get a high grade on this assignment will require a particularly efficient implementation value iteration with intelligent choice of parameters, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

Note that you can write your agent to support modified versions of the game, which is also demonstrated below. For submissions that support this, the tests will include modified versions of the game, some where the game is *more obvious*, and some where it is *less obvious* than the default rules. Getting high marks requires supporting this feature. For those who are struggling to write a good agent, supporting this feature will result in additional tests which are more forgiving, so this might be an attractive option.

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (the report), but only the code in this notebook will be used in the automated testing.

## Dice Game Class
A class called `DiceGame` is provided within `dice_game.py` for you to use in your solution. Here is a demonstration of its features.

When you create a DiceGame object, by default you will get the rules as stated above. 
```python
game = DiceGame()
```
Creates a game with 3 normal 6-sided dice. 

You may wish to modify the game mechanics, and you can do this using the other constructor arguments, for example:
```python
game = DiceGame(dice=4, sides=3, values=[1, 2, 6], bias=[0.1, 0.1, 0.8], penalty=2)
```
will create a game where you roll 4 dice, each with 3 sides, labelled 1, 2, and 6, where each die is far more likely to roll a 6 than they are to roll a 1 or a 2, and furthermore the penalty for rerolling is now 2 points instead of 1. *Note: this does not necessarily result in an interesting game.*

In games with unusual values or sides (3-sided dice are unusual without trying to turn them upside down), when there are duplicates, `value[i]` becomes `value[-i]`. With odd-sided dice, the middle value will flip onto itself.

Once created, the `DiceGame` object can be run in two different modes, *simulation* and *analysis*. It is likely that you will mostly use [*analysis* mode](#Analysis-Mode) to derive your agent's behaviour, but either way some understanding of simulation mode might be useful.

### Simulation Mode
The object provides the methods required to simulate playing the game. This might be useful if you want to test your agent, or you just want to try playing the game yourself, as we did in the cell earlier. The current dice values are found by calling `get_dice_state()`, they will always be listed in ascending order. 

In [24]:
from dice_game import DiceGame
from IPython.display import HTML,Javascript, display

import numpy as np
import time

# setting a seed for the random number generator gives repeatable results, making testing easier!
np.random.seed(111)

game = DiceGame()
game.get_dice_state()

(2, 3, 4)

To roll the dice, you call the `roll` method which takes one parameter: a tuple representing which dice you want to hold, numbered from zero. We rolled (2, 3, 4). Suppose we want to hold the 2, we would pass the tuple `(0,)` into the `roll` method (note we need to include the comma so that Python knows this is a tuple).

The `roll` method returns a tuple containing: the reward for this action, the new state, and whether the game is over.

In [25]:
reward, new_state, game_over = game.roll((0,))
print(reward)
print(new_state)
print(game_over)

-1
(2, 2, 5)
False


Now suppose we are happy and wish to stick to get our final score. We can call the `roll` method and supply a tuple containing all three dice.

In [26]:
reward, new_state, game_over = game.roll((0, 1, 2))
print(reward)
print(new_state)
print(game_over)

15
(5, 5, 5)
True


Remember that the return value is just the reward for the action, in this case 15. To get our final score we can inspect `game.score`. We rerolled once, so expect to get a score of 14.

In [27]:
print(game.score)

14


And to play again we can call `game.reset()` which returns the new starting dice.

In [28]:
game.reset()

(1, 1, 3)

### Analysis Mode
In analysis mode, you are not playing the game, but asking the object for *all possible outcomes of certain actions*.

First of all, it is useful to know that all the possible states and all the possible actions are stored inside the object. Try changing the game mechanics on the first line (e.g. add `dice=2` to the constructor) and see how the other information is updated.

In [29]:
game = DiceGame()
print(f"The first 5 of {len(game.states)} possible dice rolls are: {game.states[0:5]}")
print(f"The possible actions on any given turn are: {game.actions}")

The first 5 of 56 possible dice rolls are: [(1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 1, 4), (1, 1, 5)]
The possible actions on any given turn are: [(), (0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2)]


Finally, the most important method is `get_next_states(action, dice_state)`. This allows you to get all the possible resulting states for any given state and action.

Earlier we had the roll of `(2, 3, 4)` and decided to hold the 2. The game can calculate all possible outcomes for us, and crucially will also give us the probability of each state occurring.

In [30]:
game = DiceGame()
states, game_over, reward, probabilities = game.get_next_states((0,), (2, 3, 4))
for state, probability in zip(states, probabilities):
    print(f"Would get roll of {state} with probability {probability}")

Would get roll of (1, 1, 2) with probability 0.02777777777777778
Would get roll of (1, 2, 2) with probability 0.055555555555555566
Would get roll of (1, 2, 3) with probability 0.055555555555555566
Would get roll of (1, 2, 4) with probability 0.055555555555555566
Would get roll of (1, 2, 5) with probability 0.055555555555555566
Would get roll of (1, 2, 6) with probability 0.05555555555555559
Would get roll of (2, 2, 2) with probability 0.02777777777777778
Would get roll of (2, 2, 3) with probability 0.055555555555555566
Would get roll of (2, 2, 4) with probability 0.055555555555555566
Would get roll of (2, 2, 5) with probability 0.055555555555555566
Would get roll of (2, 2, 6) with probability 0.05555555555555559
Would get roll of (2, 3, 3) with probability 0.02777777777777778
Would get roll of (2, 3, 4) with probability 0.055555555555555566
Would get roll of (2, 3, 5) with probability 0.055555555555555566
Would get roll of (2, 3, 6) with probability 0.05555555555555559
Would get roll o

The method also works consistently when all dice are held, reporting that this action would cause the game to be over and giving the reward. Note that the list of states returned contains the value `None`. This is to denote that the game has entered a terminal state – no further actions would be allowed. The game does not return the final dice here, because that is another valid state (from which there would still be actions available). Also, the `reward` value is not the same as the final `score` of any given game, because it does not include any possible previous penalties for rerolling.

In [31]:
states, game_over, reward, probabilities = game.get_next_states((0, 1, 2), (2, 2, 5))
print(states)
print(game_over)
print(reward)

[None]
True
15


## Part One
You should write all of your code for your dice game agent below this cell.

Let's start with some example agents, so you can see the format we will use. In the cell below are two agents which do not play particularly well. One always holds immediately, the other will keep re-rolling all dice until they get the best possible dice (`(1, 1, 1)` or `(1, 1, 6)`), ignoring the massive penalty this will incur from re-rolling. Neither of them is considering the probabilities involved in the game.

There is also a function which will run the game with an instance of a given agent. When you run the cell, it will simulate a game with each agent.

In [32]:
from abc import ABC, abstractmethod


class DiceGameAgent(ABC):
    def __init__(self, game):
        self.game = game
    
    @abstractmethod
    def play(self, state):
        pass


class AlwaysHoldAgent(DiceGameAgent):
    def play(self, state):
        return (0, 1, 2)


class PerfectionistAgent(DiceGameAgent):
    def play(self, state):
        if state == (1, 1, 1) or state == (1, 1, 6):
            return (0, 1, 2)
        else:
            return ()
        
        
def play_game_with_agent(agent, game, verbose=False):
    state = game.reset()
    
    if(verbose): print(f"Testing agent: \n\t{type(agent).__name__}")
    if(verbose): print(f"Starting dice: \n\t{state}\n")
    
    game_over = False
    actions = 0
    while not game_over:
        action = agent.play(state)
        actions += 1
        
        if(verbose): print(f"Action {actions}: \t{action}")
        _, state, game_over = game.roll(action)
        if(verbose and not game_over): print(f"Dice: \t\t{state}")

    if(verbose): print(f"\nFinal dice: {state}, score: {game.score}")
        
    return game.score


if __name__ == "__main__":
    # random seed makes the results deterministic
    # change the number to see different results
    # or delete the line to make it change each time it is run
    np.random.seed(1)
    
    game = DiceGame()
    
    agent1 = AlwaysHoldAgent(game)
    play_game_with_agent(agent1, game, verbose=True)
    
    print("\n")
    
    agent2 = PerfectionistAgent(game)
    play_game_with_agent(agent2, game, verbose=True)

Testing agent: 
	AlwaysHoldAgent
Starting dice: 
	(1, 1, 2)

Action 1: 	(0, 1, 2)

Final dice: (2, 6, 6), score: 14


Testing agent: 
	PerfectionistAgent
Starting dice: 
	(2, 3, 3)

Action 1: 	()
Dice: 		(3, 4, 5)
Action 2: 	()
Dice: 		(1, 2, 6)
Action 3: 	()
Dice: 		(3, 4, 5)
Action 4: 	()
Dice: 		(1, 2, 5)
Action 5: 	()
Dice: 		(2, 5, 6)
Action 6: 	()
Dice: 		(1, 6, 6)
Action 7: 	()
Dice: 		(1, 2, 6)
Action 8: 	()
Dice: 		(1, 3, 6)
Action 9: 	()
Dice: 		(2, 4, 5)
Action 10: 	()
Dice: 		(1, 5, 6)
Action 11: 	()
Dice: 		(5, 5, 6)
Action 12: 	()
Dice: 		(1, 2, 5)
Action 13: 	()
Dice: 		(2, 3, 6)
Action 14: 	()
Dice: 		(1, 1, 2)
Action 15: 	()
Dice: 		(2, 2, 5)
Action 16: 	()
Dice: 		(1, 3, 4)
Action 17: 	()
Dice: 		(1, 4, 5)
Action 18: 	()
Dice: 		(1, 3, 5)
Action 19: 	()
Dice: 		(1, 3, 4)
Action 20: 	()
Dice: 		(4, 4, 6)
Action 21: 	()
Dice: 		(1, 4, 6)
Action 22: 	()
Dice: 		(1, 3, 5)
Action 23: 	()
Dice: 		(1, 3, 6)
Action 24: 	()
Dice: 		(5, 5, 6)
Action 25: 	()
Dice: 		(3, 4, 5)
Ac

In the cell below, write the logic for your own agent. To do this, you need to write a class named `MyAgent` which is a subclass of `DiceGameAgent`, which overrides the `play(self, state)` method. There is some skeleton code in the cell below to help you get started. This is the code which will be tested.

In [33]:
import numpy as np

class DiceGameAgent:
    def __init__(self, game):
        self.game = game
        self.state_values = {}
        self.best_actions = {}
        self.discount_factor = 0.9  # Discount factor for future rewards
        self.convergence_threshold = 1e-4  # Threshold to stop value iteration
        self.perform_value_iteration()

    def perform_value_iteration(self):
        # Initialize value function
        for state in self.game.states:
            self.state_values[state] = 0

        while True:
            max_change = 0
            for state in self.game.states:
                old_value = self.state_values[state]
                max_value = float('-inf')
                
                for action in self.game.actions:
                    expected_reward = 0
                    next_states, _, reward, probabilities = self.game.get_next_states(action, state)
                    
                    for next_state, prob in zip(next_states, probabilities):
                        if next_state is not None:
                            expected_reward += prob * (reward + self.discount_factor * self.state_values[next_state])
                        else:
                            expected_reward += prob * reward
                    
                    if expected_reward > max_value:
                        max_value = expected_reward
                        self.best_actions[state] = action
                
                self.state_values[state] = max_value
                max_change = max(max_change, abs(old_value - max_value))
            
            if max_change < self.convergence_threshold:
                break

    def play(self, state):
        return self.best_actions[state]

# Assuming DiceGame class and play_game_with_agent function are defined as in your notebook
if __name__ == "__main__":
    np.random.seed(1)
    game = DiceGame(dice=2, sides=3)  # Adjusted for extended rules
    agent = DiceGameAgent(game)
    play_game_with_agent(agent, game, verbose=True)

Testing agent: 
	DiceGameAgent
Starting dice: 
	(1, 1)

Action 1: 	(0, 1)

Final dice: (3, 3), score: 6


You're free to modify the structure of the code above as you like, but you must make sure your code is compatible with our testing framework, which you can check below.

All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
Your agent will be tested in two ways. First it will be tested in actual random games, and the average score will be measured. All students will get the exact same dice rolls, so the best strategies will get the most points. Second, it will be analysed in specific circumstances (i.e. specific dice rolls) to test what it does compared to the optimal strategy.

In addition, if your code supports it, we will repeat these tests with *extended rules*, i.e. not using the default game with three 6-sided fair dice. You can read more about how this affects grading in [this section](#Choice-of-Algorithm). 

The tests run with a timeout to prevent any submissions slowing down the entire grading. On the testing machine, your agent must take less than 30 seconds to construct, and less than 2 seconds to produce each action. For high marks, execution time will also factor into grading (both construction and average per move time).

As we have said before, the best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
Run the following cell to test your code 10 times and print the average score. You should run this test to ensure your code is working correctly. The actual tests which determine your grade will be different and more thorough. 

To enable the tests, set the constant `SKIP_TESTS` to `False`.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [34]:
SKIP_TESTS = True

if not SKIP_TESTS:

    total_score = 0
    total_time = 0
    n = 10

    np.random.seed()

    print("Testing basic rules.")
    print()

    _game = DiceGame()

    start_time = time.process_time()
    test_agent = MyAgent(_game)
    total_time += time.process_time() - start_time

    for i in range(n):
        start_time = time.process_time()
        score = play_game_with_agent(test_agent, _game)
        total_time += time.process_time() - start_time

        print(f"Game {i} score: {score}")
        total_score += score

    print()
    print(f"Average score: {total_score/n}")
    print(f"Total time: {total_time:.4f} seconds")

#### Opt-In For Extended Rules
If you wish your code to be tested with rules other than the defaults, set the constant `TEST_EXTENDED_RULES` to `True` on the next line. Another test will be performed to check your code still works. If you get an error, you are likely not supporting the extended rules properly.

Refer to [this section](#Choice-of-Algorithm) to understand more about how extended rules factor into your possible grade.

**Note:** you need to have `SKIP_TESTS` set to `False` in the cell above (and run it!) to enable the tests below. The value of `TEST_EXTENDED_RULES` *will* be used in the grading tests.

In [35]:
TEST_EXTENDED_RULES = True

if not SKIP_TESTS and TEST_EXTENDED_RULES:
    total_score = 0
    total_time = 0
    n = 10

    print("Testing extended rules – two three-sided dice.")
    print()

    _game = DiceGame(dice=2, sides=3)

    start_time = time.process_time()
    test_agent = MyAgent(_game)
    total_time += time.process_time() - start_time

    for i in range(n):
        start_time = time.process_time()
        score = play_game_with_agent(test_agent, _game)
        total_time += time.process_time() - start_time

        print(f"Game {i} score: {score}")
        total_score += score

    print()
    print(f"Average score: {total_score/n}")
    print(f"Average time: {total_time/n:.5f} seconds")

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [36]:
import sys
import pathlib

fail = False;

p1 = pathlib.Path('./report.pdf')
p2 = pathlib.Path('./Report.pdf')

success = '\033[1;32m[✓]\033[0m'
issue = '\033[1;33m[!]'
error = '\033[1;31m\t✗'

#######
##
## Skip Game check.
##
## Test to ensure the SKIP_GAME variable is set to True to prevent it slowing down the automarker.
##
#######

if not SKIP_GAME:
    fail = True;
    print("{} \'SKIP_GAME\' is incorrectly set to False.\033[0m".format(issue))
    print("{} You must set the SKIP_GAME constant to True at the top of this notebook.\033[0m".format(error))
else:
    print('{} \'SKIP_GAME\' is set to true.\033[0m'.format(success))


#######
##
## Skip Tests check.
##
## Test to ensure the SKIP_TESTS variable is set to True to prevent it slowing down the automarker.
##
#######

if not SKIP_TESTS:
    fail = True;
    print("{} \'SKIP_TESTS\' is incorrectly set to False.\033[0m".format(issue))
    print("{} You must set the SKIP_TESTS constant to True in the cell above.\033[0m".format(error))
else:
    print('{} \'SKIP_TESTS\' is set to true.\033[0m'.format(success))

#######
##
## Report File Check.
##
## Test that checks there is a report pdf file found in the same folder as the notebook. This is required by the coursework specification.
##
#######

if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("{} Report PDF not found.\033[0m".format(issue))
    print("{} You must include a separate file called report.pdf in your submission.\033[0m".format(error))
else:
    print('{} Report PDF found.\033[0m'.format(success))

#######
##
## File Name check.
##
## Test to ensure file has the correct name. This is important for the marking system to correctly process the submission.
##
#######
    
p3 = pathlib.Path('./dicegame.ipynb')
if not p3.is_file():
    fail = True
    print("{} The notebook name is incorrect.\033[0m".format(issue))
    print("{} This notebook file must be named dicegame.ipynb\033[0m".format(error))
else:
    print('{} The notebook name is correct.\033[0m'.format(success))

#######
##
## Test set check.
##
## Test that checks your classifier actually works. The calls made here are the same made by the automarker - albeit with different data. If your work fails this test it will score 0 in the automarker.
##
#######

try:
    if "MyAgent" not in dir():
        fail = True;
        print("{} MyAgent test failed - No such class MyAgent.\033[0m".format(issue))
        print("{} You must include a class called MyAgent as defined above.\033[0m".format(error))
        
        if TEST_EXTENDED_RULES:
            print("{} TEST_EXTENDED_RULES are ON - Failed".format(issue))
            print("{} No such class MyAgent.\033[0m".format(error))
            print("{} You must include a class called MyAgent as defined above.\033[0m".format(error))
        else:
            print("{} TEST_EXTENDED_RULES are OFF".format(sucess))
            
    else:    
        game = DiceGame()
        agent = MyAgent(game)
        action = agent.play((1, 1, 1))

        if action not in game.actions:
            print("{} MyAgent test warning - Invalid Actions.\033[0m".format(issue))
            print("{} Your agent does not seem to produce a valid action with the default rules.\033[0m".format(error))
            print()
            print("{} Your assignment is unlikely to get any marks from the autograder. While we will\033[0m".format(error))
            print("{} try to check it manually to assign some partial credit, we encourage you to ask\033[0m".format(error))
            print("{} for help on the forum or directly to a tutor.\033[0m".format(error))
            print()
            print("{} Please use the report file to explain your code anyway.".format(error))
        else:
            print("{} MyAgent test success - My agent was able to run successfully.\033[0m".format(success))

        if TEST_EXTENDED_RULES:
            game = DiceGame(dice=2, sides=8)
            agent = MyAgent(game)
            try:
                action = agent.play((7, 8))
            except:
                action = None

            if action not in game.actions:
                fail = True
                print("{} TEST_EXTENDED_RULES is ON - Failed.\033[0m".format(issue))
                print("{} Your agent does not produce a valid action with the extended rules.\033[0m".format(error))
                print("{} Turn off TEST_EXTENDED_RULES if you cannot fix this error.\033[0m".format(error))
            else:
                print("{} TEST_EXTENDED_RULES are ON - Success (extended rules will be tested).\033[0m".format(success))
        else:
            print("{} \033[1;31mTEST_EXTENDED_RULES are OFF (extended rules will not be tested).\033[0m".format(success))
            print("\t\033[1;31mYou can submit your work like this, however, you will not be able to get the maximum marks.\033[0m")
            print("\t\033[1;31mFor more infomration please read the extended rules description above.\033[0m")
except:
    sys.stderr.write("Error running functional test.")

#######
##
## Final Summary
##
## Prints the final results of the submission tests.
##
#######

if fail:
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")

else:
    print("\033[1m\n\n")
    print("╔═══════════════════════════════════════════════════════════════╗")
    print("║                        Congratulations!                       ║")
    print("║                                                               ║")
    print("║            Your work meets all the required criteria          ║")
    print("║                   and is ready for submission.                ║")
    print("╚═══════════════════════════════════════════════════════════════╝")
    print("\033[0m")
    

[✓] 'SKIP_GAME' is set to true.
[✓] 'SKIP_TESTS' is set to true.
[!] Report PDF not found.
	✗ You must include a separate file called report.pdf in your submission.
[✓] The notebook name is correct.
[✓] MyAgent test success - My agent was able to run successfully.
[✓] TEST_EXTENDED_RULES are ON - Success (extended rules will be tested).


Your submission is not ready! Please read and follow the instructions above.

In [37]:
# This is a TEST CELL. Do not delete or change.